In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
import time

%matplotlib inline

df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[:100]

date_t = dt.today().strftime("%Y%m%d")

#https://foursquare.com/developers/apps 申请网站
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' 
Paw = 'K13KOCRW22OVDQJBMYNBD5B4EMWBPBLU5CCDX4UWVL3V3T03'

def getNearbyPlaces(names, latitudes, longitudes, date_t, radius = 500, limit = 100,):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        
        number_food = requests.get(url+'&section=food').json()['response']['totalResults']
        number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']
        
        number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
        number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
        number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']
        
        url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        price_low = requests.get(url_price_low).json()['response']['totalResults']
        
        url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
            ID, Paw, date_t, lat, lng, radius, limit)
        price_high = requests.get(url_price_high).json()['response']['totalResults']
        
        venues_list.append([(name, lat, lng, 
            number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])
                            
                            
    selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
    return(selected_ven)

df_venues = getNearbyPlaces(names = df_lat_long['street_name'],
                            latitudes = df_lat_long['Latitude'],
                            longitudes = df_lat_long['Longitude'],
                            date_t = dt.today().strftime("%Y%m%d"))

In [1]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from sklearn import preprocessing
%matplotlib inline

df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[:100]

date_t = dt.today().strftime("%Y%m%d")

#https://foursquare.com/developers/apps 申请网站
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' 
Paw = 'K13KOCRW22OVDQJBMYNBD5B4EMWBPBLU5CCDX4UWVL3V3T03'

venues_list=[]
names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']
date_t = dt.today().strftime("%Y%m%d")
radius = 500
limit = 100
    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']

In [3]:
selected_ven.to_csv('selected_ven1.csv')

In [6]:
selected_ven1 = pd.read_csv('selected_ven1.csv')
selected_ven1

,Unnamed: 0,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
0,0,ANG MO KIO AVE 10,1.367602,103.856844,23,18,23,5,17,8,2
1,1,ANG MO KIO AVE 4,1.382319,103.838706,10,5,13,6,11,5,0
2,2,ANG MO KIO AVE 5,1.376825,103.847556,12,13,18,4,10,1,1
3,3,ANG MO KIO AVE 1,1.364355,103.843642,15,12,9,4,17,6,0
4,4,ANG MO KIO AVE 3,1.369691,103.840906,19,12,11,7,15,6,1
...,...,...,...,...,...,...,...,...,...,...,...
95,95,WATERLOO ST,1.299426,103.852085,79,102,136,62,87,26,5
96,96,NEW MKT RD,1.287705,103.845216,130,131,65,18,126,35,16
97,97,TANJONG PAGAR PLAZA,1.275570,103.842993,88,102,65,10,104,33,11
98,98,QUEEN ST,1.302628,103.856076,118,115,113,24,106,38,8


In [12]:

df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[100:200]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven2 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven2.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven2.to_csv('selected_ven2.csv')

In [17]:
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[200:300]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven3 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven3.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven3.to_csv('selected_ven3.csv')

In [33]:
ID = '2QT0EPAJ0W3WJDRIWKVQNNNDEX5Q2WO5FAZK4ZAVASDGNTF5'
Paw = '5MIIJM2R1Z544OFNQ2QKJB3DCFXU0V2XRTH2K1J2S3UGXCUM'
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[361:461]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven4 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven4.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven4.to_csv('selected_ven4.csv')

KeyError: 'totalResults'

In [40]:
ID = 'I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG' 
Paw = 'L3HUMVJA3RAKEYFTOQF5YLGOKINC502MMFFMQGKT3S4H3E3P'


df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long = df_lat_long[411:]

names = df_lat_long['street_name']
latitudes = df_lat_long['Latitude']
longitudes = df_lat_long['Longitude']

    
for name, lat, lng in zip(names, latitudes, longitudes):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        ID, Paw, date_t, lat, lng, radius, limit)

    number_food = requests.get(url+'&section=food').json()['response']['totalResults']
    number_drink = requests.get(url+'&section=drinks').json()['response']['totalResults'] + requests.get(url+'&section=coffee').json()['response']['totalResults']

    number_shop = requests.get(url+'&section=shops').json()['response']['totalResults']
    number_art = requests.get(url+'&section=arts').json()['response']['totalResults']
    number_trending = requests.get(url+'&section=trending').json()['response']['totalResults']

    url_price_low = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=1'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_low = requests.get(url_price_low).json()['response']['totalResults']

    url_price_high = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&price=3,4'.format(
        ID, Paw, date_t, lat, lng, radius, limit)
    price_high = requests.get(url_price_high).json()['response']['totalResults']

    venues_list.append([(name, lat, lng, 
        number_food,number_drink,number_shop,number_art,number_trending,price_low,price_high)])


selected_ven5 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven5.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven5.to_csv('selected_ven5.csv')

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=I2E3FGJX0Z12Y0PHJ22W3WZBSUNDYWPJAQCFMSHJQBSAQJKG&client_secret=L3HUMVJA3RAKEYFTOQF5YLGOKINC502MMFFMQGKT3S4H3E3P&v=20210221&ll=1.4446381,103.8291537&radius=500&limit=100&section=food (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe4f6951bb0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [22]:
venues_list

[[('ANG MO KIO AVE 10', 1.3676017, 103.8568442, 23, 18, 23, 5, 17, 8, 2)],
 [('ANG MO KIO AVE 4', 1.3823185, 103.8387061, 10, 5, 13, 6, 11, 5, 0)],
 [('ANG MO KIO AVE 5', 1.3768247, 103.8475557, 12, 13, 18, 4, 10, 1, 1)],
 [('ANG MO KIO AVE 1', 1.3643546, 103.8436418, 15, 12, 9, 4, 17, 6, 0)],
 [('ANG MO KIO AVE 3', 1.3696908, 103.8409057, 19, 12, 11, 7, 15, 6, 1)],
 [('ANG MO KIO AVE 9', 1.3827747, 103.8394374, 12, 9, 12, 6, 11, 5, 0)],
 [('ANG MO KIO AVE 8', 1.3784539, 103.84742, 11, 10, 7, 4, 12, 1, 0)],
 [('ANG MO KIO AVE 6', 1.3832578, 103.8441199, 13, 12, 6, 10, 13, 4, 0)],
 [('ANG MO KIO ST 52', 1.3730573, 103.8507324, 32, 9, 59, 3, 30, 7, 1)],
 [('BEDOK NORTH AVE 4', 1.3340399, 103.9414261, 15, 10, 11, 5, 15, 11, 1)],
 [('BEDOK NORTH AVE 1', 1.3297469, 103.9264326, 28, 21, 14, 13, 18, 9, 2)],
 [('BEDOK NORTH RD', 1.3313413, 103.9337502, 19, 17, 22, 17, 21, 12, 0)],
 [('BEDOK STH AVE 1', 1.314061, 103.9336214, 6, 1, 3, 1, 6, 0, 0)],
 [('BEDOK RESERVOIR RD', 1.3296754, 103.904835

In [35]:
selected_ven4 = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
selected_ven4.columns = ['street_name', 'Latitude', 'Longitude', 'Food', 'Drink', 'Shope', 'Art', 'Trending', 'Price_low', 'Price_high']
selected_ven4.to_csv('selected_ven4.csv')

In [41]:
selected_ven4

,street_name,Latitude,Longitude,Food,Drink,Shope,Art,Trending,Price_low,Price_high
0,ANG MO KIO AVE 10,1.367602,103.856844,23,18,23,5,17,8,2
1,ANG MO KIO AVE 4,1.382319,103.838706,10,5,13,6,11,5,0
2,ANG MO KIO AVE 5,1.376825,103.847556,12,13,18,4,10,1,1
3,ANG MO KIO AVE 1,1.364355,103.843642,15,12,9,4,17,6,0
4,ANG MO KIO AVE 3,1.369691,103.840906,19,12,11,7,15,6,1
...,...,...,...,...,...,...,...,...,...,...
444,HOLLAND CL,1.307305,103.796033,68,38,60,4,54,25,1
445,TANGLIN HALT RD,1.300528,103.798360,35,24,26,9,23,11,0
446,COMMONWEALTH AVE,1.302493,103.798062,36,24,23,8,25,11,0
447,WELLINGTON CIRCLE,1.452079,103.820354,29,14,33,6,24,10,0


In [46]:
df_lat_long = pd.read_csv("df_lat_long.csv")
df_lat_long[df_lat_long['street_name']=='CANBERRA LINK']

,Unnamed: 0,street_name,Latitude,Longitude
411,411,CANBERRA LINK,1.444638,103.829154


In [48]:
df_lat_long[411:]

,Unnamed: 0,street_name,Latitude,Longitude
411,411,CANBERRA LINK,1.444638,103.829154
412,412,SENGKANG WEST AVE,1.391518,103.882569
413,413,SENGKANG EAST RD,1.382487,103.889071
414,414,SERANGOON CENTRAL DRIVE,1.354677,103.871602
415,415,SERANGOON AVE 3,1.352922,103.870596
...,...,...,...,...
564,564,JALAN MEMBINA BARAT,1.283347,103.826121
565,565,BUANGKOK STH FARMWAY 1,1.388388,103.987492
566,566,ALEXANDRA RD,1.273529,103.802049
567,567,SEMBAWANG RD,1.453349,103.831769
